In [1]:
!rm -rf sample_data

# VGG shape 31x128
!wget http://dept-info.labri.fr/~hanna/ProjetClassif/train_id_genres_vgg.pickle

!wget http://dept-info.labri.fr/~hanna/ProjetClassif/test_id_vgg.pickle

# Docs
# https://github.com/tensorflow/models/tree/master/research/audioset/vggish

--2022-12-19 18:33:08--  http://dept-info.labri.fr/~hanna/ProjetClassif/train_id_genres_vgg.pickle
Resolving dept-info.labri.fr (dept-info.labri.fr)... 147.210.9.83, 2001:660:6101:404::2:80
Connecting to dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dept-info.labri.fr/~hanna/ProjetClassif/train_id_genres_vgg.pickle [following]
--2022-12-19 18:33:09--  https://dept-info.labri.fr/~hanna/ProjetClassif/train_id_genres_vgg.pickle
Connecting to dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63618706 (61M)
Saving to: ‘train_id_genres_vgg.pickle’

train_id_genres_vgg 100%[===================>]  60.67M  38.2MB/s    in 1.6s    

2022-12-19 18:33:10 (38.2 MB/s) - ‘train_id_genres_vgg.pickle’ saved [63618706/63618706]

URL transformed to HTTPS due to an HSTS policy
--2022-12-19 18:33:10--  https://dept-info.labri.fr

In [2]:
import pickle

# Train dataset : list of (id, genre, embed)
data = pickle.load(open('train_id_genres_vgg.pickle','rb'))

ids = [x[0] for x in data]
genres = [x[1] for x in data]
embed = [x[2] for x in data]

#one hot encoding
import tensorflow as tf
genres = [int(x)-1 for x in genres]
genres = tf.keras.utils.to_categorical(genres)

# Example
TRACK_NUMBER = 10
print("track n°", TRACK_NUMBER)
print("Song genres ", genres[TRACK_NUMBER])
print("Embed shape ", embed[TRACK_NUMBER].shape)
print("Embed ", embed[TRACK_NUMBER])

import numpy as np
x_train = np.array(embed)
y_train = np.array(genres)
print(x_train.shape, y_train.shape)

# Test dataset : list of (id, embed)
data = pickle.load(open('test_id_vgg.pickle','rb'))

tmp = [(x[0],x[1]) for x in data if len(x[1]) == 31]
ids = [x[0] for x in tmp]
embed = [x[1] for x in tmp]

# Example
TRACK_NUMBER = 10
print("track n°", TRACK_NUMBER)
print("Song id ", ids[TRACK_NUMBER])
print("Embed shape ", embed[TRACK_NUMBER].shape)
print("Embed ", embed[TRACK_NUMBER])

import numpy as np
embed = np.array(embed)
ids = np.array(ids)
print(embed.shape, ids.shape)

track n° 10
Song genres  [0. 0. 1. 0. 0. 0. 0. 0.]
Embed shape  (31, 128)
Embed  [[ 4.9715251e-02  6.4623654e-03  1.4974146e-01 ...  2.5808668e-01
   1.7034711e-01  1.9361663e-01]
 [-1.7946199e-01  4.0544292e-01 -1.4402509e-01 ... -6.0221761e-02
   2.1952584e-02  3.1505102e-01]
 [-5.2178204e-01  4.4526982e-01  7.8287050e-02 ... -4.9954882e-01
   1.7422524e-01  3.1353217e-01]
 ...
 [ 1.2343328e-01  7.0264429e-02  4.1037798e-05 ...  2.9932988e-01
   3.0311149e-01 -1.6516665e-01]
 [ 2.5620168e-01 -4.7355741e-02  2.1767169e-02 ...  3.3860534e-01
   1.7308064e-02  4.5191109e-02]
 [ 5.6809157e-02  2.3029198e-01 -3.4522653e-02 ...  6.7807639e-01
   9.9629365e-02 -9.4960511e-02]]
(3995, 31, 128) (3995, 8)
track n° 10
Song id  000534
Embed shape  (31, 128)
Embed  [[-0.33152917 -0.39168087 -0.04646578 ... -0.71700513 -0.12104972
  -0.09989479]
 [-0.63577807 -0.34468648 -0.17933583 ... -1.322129   -0.10069139
  -0.07899085]
 [-0.4593254  -0.27077848 -0.01870891 ... -0.67547345 -0.09902768
  -0.10

## **VGG**


In [3]:
!pip install spec-augment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from spec_augment import SpecAugment

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.30)

In [6]:
print('x_train.shape=', x_train.shape)
print('y_train.shape=', y_train.shape)

x_train.shape= (2796, 31, 128)
y_train.shape= (2796, 8)


In [7]:
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0], img_rows , img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows , img_cols, 1)
input_shape= (img_rows, img_cols,1)

In [8]:
print('x_train.shape=', x_train.shape)
print('y_train.shape=', y_train.shape)
print('x_test.shape=', x_test.shape)
print('y_test.shape=', y_test.shape)
print('input_shape=',input_shape)

x_train.shape= (2796, 31, 128, 1)
y_train.shape= (2796, 8)
x_test.shape= (1199, 31, 128, 1)
y_test.shape= (1199, 8)
input_shape= (31, 128, 1)


In [13]:
epochs=25
batch_size= 5

spec_augment = SpecAugment(freq_mask_param=20,
                           time_mask_param=20)

vgg_model = Sequential()

vgg_model.add(spec_augment)

vgg_model.add(Flatten())
vgg_model.add(BatchNormalization())
vgg_model.add(Dense(256, activation='relu'))
vgg_model.add(layers.Dropout(0.25))
vgg_model.add(BatchNormalization())
vgg_model.add(Dense(128, activation='relu'))

vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(8, activation='softmax'))

vgg_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
vgg_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/25
560/560 [==============================] - 5s 8ms/step - loss: 1.5282 - accuracy: 0.4521 - val_loss: 1.1614 - val_accuracy: 0.6330
Epoch 2/25
560/560 [==============================] - 4s 8ms/step - loss: 1.2829 - accuracy: 0.5522 - val_loss: 1.1162 - val_accuracy: 0.6564
Epoch 3/25
560/560 [==============================] - 4s 8ms/step - loss: 1.2486 - accuracy: 0.5676 - val_loss: 1.0447 - val_accuracy: 0.6731
Epoch 4/25
560/560 [==============================] - 4s 8ms/step - loss: 1.1872 - accuracy: 0.5923 - val_loss: 1.0309 - val_accuracy: 0.6856
Epoch 5/25
560/560 [==============================] - 5s 8ms/step - loss: 1.2146 - accuracy: 0.5783 - val_loss: 1.0052 - val_accuracy: 0.6739
Epoch 6/25
560/560 [==============================] - 4s 8ms/step - loss: 1.2064 - accuracy: 0.5798 - val_loss: 1.0026 - val_accuracy: 0.6772
Epoch 7/25
560/560 [==============================] - 4s 8ms/step - loss: 1.1754 - accuracy: 0.6019 - val_loss: 1.0474 - val_accuracy: 0.6822
Epoch 

In [14]:
# Evaluate the model
print("Evaluation data")
results = vgg_model.evaluate(x_test, y_test, batch_size=5)
print("test loss, test acc:", results)

Evaluation data
240/240 [==============================] - 0s 2ms/step - loss: 1.0027 - accuracy: 0.6906
test loss, test acc: [1.0027046203613281, 0.6905754804611206]


In [15]:
embed = embed.reshape(embed.shape[0], img_rows, img_cols, 1)
print(embed.shape)

(4003, 31, 128, 1)


In [16]:
predicted_y = vgg_model.predict(embed)
predicted_y = np.argmax(predicted_y,axis=1)
print(ids.shape)
print(predicted_y.shape)

import csv
with open('vggish.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(['track_id']+['genre_id'])
        for ind,x in np.ndenumerate(ids):
          csvfile.write("%s,%s\n"%(x,predicted_y[ind]+1))
        csvfile.write("%s,%s\n"%('098559',2))
        csvfile.write("%s,%s\n"%('098565',2))
        csvfile.write("%s,%s\n"%('098568',2))
        csvfile.write("%s,%s\n"%('098569',2))
        csvfile.write("%s,%s\n"%('098571',2))

126/126 [==============================] - 0s 3ms/step
(4003,)
(4003,)
